In [9]:
import pandas as pd
from sqlalchemy import create_engine
from geopy.geocoders import Nominatim
import time
from pprint import pprint
import geopy.distance
from random import random

In [10]:
disaster_data = "static/data/disastersummary10yrs.csv"
disaster_df = pd.read_csv(disaster_data)
disaster_df.head()

,femaDeclarationString,disasterNumber,stateName,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,...,disasterCloseoutDate,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,hash,lastRefresh,id,countyState
0,DR-1943-NY,1943,New York,NY,DR,2010-10-14T16:50:00.000Z,2011,Tornado,"SEVERE STORMS, TORNADOES, AND STRAIGHT-LINE WINDS",0,...,2018-09-07T23:59:00.000Z,36,47,99047,Kings County,10116,e4ad01761a67dbf8d338e20afbf534b96c7ecdf4,2021-07-13T23:01:50.453Z,60c3b84ba0ee349d7109443a,"Kings County, New York"
1,DR-1943-NY,1943,New York,NY,DR,2010-10-14T16:50:00.000Z,2011,Tornado,"SEVERE STORMS, TORNADOES, AND STRAIGHT-LINE WINDS",0,...,2018-09-07T23:59:00.000Z,36,81,99081,Queens County,10116,b11527168cdd78c43abef2100a14f21d1070cb18,2021-07-13T23:01:50.454Z,60c3b84ba0ee349d7109443f,"Queens County, New York"
2,DR-1943-NY,1943,New York,NY,DR,2010-10-14T16:50:00.000Z,2011,Tornado,"SEVERE STORMS, TORNADOES, AND STRAIGHT-LINE WINDS",0,...,2018-09-07T23:59:00.000Z,36,85,99085,Richmond County,10116,3eaf6112364b37342ac8964f17c715830d135ec0,2021-07-13T23:01:50.455Z,60c3b84ba0ee349d7109446c,"Richmond County, New York"
3,DR-1952-CA,1952,California,CA,DR,2011-01-26T15:00:00.000Z,2011,Flood,"SEVERE WINTER STORMS, FLOODING, AND DEBRIS AND...",0,...,NaN,6,27,99027,Inyo County,11002,9e2bf971b5da4473de9217c3e7966face3c9021e,2021-07-13T23:01:50.493Z,60c3b84ba0ee349d710945b9,"Inyo County, California"
4,DR-1952-CA,1952,California,CA,DR,2011-01-26T15:00:00.000Z,2011,Flood,"SEVERE WINTER STORMS, FLOODING, AND DEBRIS AND...",0,...,NaN,6,29,99029,Kern County,11002,e8ac083a9d70510df8e4a983e76ad76bca401f29,2021-07-13T23:01:50.493Z,60c3b84ba0ee349d710945be,"Kern County, California"


In [11]:
disaster_summary_df = disaster_df.head(1000)

In [12]:
app = Nominatim(user_agent="tutorial")

In [13]:
def get_lat(countyState):
    try:
        lat = app.geocode(countyState).raw["lat"]
        lon = app.geocode(countyState).raw["lon"]
        #print(f'{countyState} lat = {lat}, lon = {lon}')
        return lat,lon
    except (TypeError, AttributeError):
        lat = "unknown"
        lon = "unknown"
        #print(f'{countyState} not found')
        return lat, lon
    

   
    
    

In [14]:
#get_lat("Clay County, Indiana")

In [15]:
disaster_summary_df["lat"],disaster_summary_df["lon"]  = zip(*disaster_summary_df["countyState"].apply(get_lat))

<ipython-input-15-4eeb50dc43c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disaster_summary_df["lat"],disaster_summary_df["lon"]  = zip(*disaster_summary_df["countyState"].apply(get_lat))


In [26]:
disaster_summary_df.drop(disaster_summary_df[disaster_summary_df['lat'] == "unknown"].index, inplace = True)

C:\Users\rspic\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [28]:
disaster_summary_df.to_csv("static/data/disasterData.csv")